Topic: Project 5  
Subject: Creating Final Dataframe   
Date: 12/12/2017  
Name: Zach Heick  

**Summary**: Gathers all data from multiple SQL tables and saves a dataframe for modeling.

In [14]:
import pickle
import os
import boto3
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, select
import os
import pickle
from nltk.corpus import stopwords
from nltk import RegexpTokenizer

In [15]:
engine_name = 'postgresql://' + str(os.environ['zU']) + ':' + str(os.environ['zP']) + str(os.environ['AWS_PROJECT_KOJAK_EC2'])
engine = create_engine(engine_name)

In [16]:
df = pd.read_sql_query('''SELECT 
                           tracks.album_art,
                           tracks.album_name,
                           tracks.duration_ms,
                           tracks.energy,
                           tracks.liveness,
                           tracks.loudness,
                           tracks.speechiness,
                           tracks.tempo,
                           tracks.track_id,
                           tracks.track_name,
                           artists.artist_popularity,
                           artists.artist_name,
                           artists.artist_id,
                           lyrics.lyrics 
                           FROM tracks
                           JOIN artists ON tracks.artist_id = artists.artist_id
                           JOIN lyrics ON tracks.track_id = lyrics.track_id
                           WHERE tracks.energy IS NOT NULL
                           AND lyrics.lyrics IS NOT NULL
                           AND artists.artist_name != 'Kid Rock'
                           ORDER BY tracks.track_name;''', engine)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19831 entries, 0 to 19830
Data columns (total 6 columns):
track_name           19831 non-null object
artist_name          19831 non-null object
album_art            19831 non-null object
lyrics               19831 non-null object
track_id             19831 non-null object
track_preview_url    14400 non-null object
dtypes: object(6)
memory usage: 929.7+ KB


In [18]:
import string

In [19]:
df['track_name'] = df['track_name'].apply(lambda i: string.capwords(i))

In [20]:
df.drop_duplicates(subset=['track_name', 'artist_name'], inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19831 entries, 0 to 19830
Data columns (total 6 columns):
track_name           19831 non-null object
artist_name          19831 non-null object
album_art            19831 non-null object
lyrics               19831 non-null object
track_id             19831 non-null object
track_preview_url    14400 non-null object
dtypes: object(6)
memory usage: 1.1+ MB


In [22]:
df.reset_index(drop=True, inplace=True)

In [23]:
df.drop(df[df['lyrics'].str.contains('<span')].index, inplace=True)

In [24]:
df.reset_index(drop=True, inplace=True)

In [25]:
len(df)

19831

In [26]:
with open('../Web_App/static/all_lyrics_dataframe.pickle', 'wb') as f:
    pickle.dump(df, f)

In [22]:
s3 = boto3.client('s3', 
                  aws_access_key_id=os.environ['AWS_ACCESS_KEY'], 
                  aws_secret_access_key=os.environ['AWS_SECRET_KEY'])

In [23]:
s3.upload_file('../Data_Files/all_lyrics_dataframe.pickle', 'metis-project-kojak-bucket', 'all_lyrics_dataframe.pickle')